<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/llama2_rag_7b_shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import sys
import os

In [16]:
!pip3 install virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 44.3 MB/s eta 0:00:00


In [17]:
!which pip3

/usr/local/bin/pip3


True

In [50]:
!virtualenv llama2_env --quiet

In [51]:
!source llama2_env/bin/activate

In [64]:
env_path = os.path.join("/content","llama2_env")
env_path

'/content/llama2_env'

Set up the virtual environment

In [66]:
env_path

'/content/llama2_env'

In [68]:
p = env_path + "/lib/python3.10/site-packages"
p

'/content/llama2_env/lib/python3.10/site-packages'

In [69]:
sys.path.insert(0,os.path.join(p))

In [71]:
type(sys.path)

list

In [73]:
sys.path.append(os.path.join("/content/llama2_env/bin"))

In [74]:
sys.path

['/content/llama2_env/lib/python3.10/site-packages',
 '/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/llama2_env/bin']

In [78]:
!source /content/llama2_env/bin/activate

In [75]:
os.environ["PATH"] = "/content/llama2_env/bin:" + os.environ["PATH"]

In [79]:
!which python

/content/llama2_env/bin/python


In [80]:
!pip install transformers accelerate einops langchain xformers \
bitsandbytes sentence_transformers  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [81]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
import os

In [82]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import os

In [83]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [84]:
cuda.is_available()

True

In [85]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-13b-chat-hf" , "models--meta-llama--Llama-2-13b-chat-hf", "snapshots", "1")
os.path.exists(model_id)

True

In [86]:
device = f'cuda:{cuda.current_device()}'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [26]:
!pip install accelerate --quiet

In [87]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Prepare model, tokenizer: 416.788 sec.


In [88]:
print(model.get_memory_footprint())

7134302208


In [89]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [93]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import  ConversationSummaryMemory

In [90]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

In [91]:
questions = [ "Write me a detailed scenario for a Nashville songwriter.",

"Now identify one person in this scenario who you might meet in the BlueBird Cafe. ",
"Write me three short stories that from events in the scenario.",
"Add dates and times for the reports." ]

In [94]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are an intelligence analyst}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt  = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

In [95]:
memory = ConversationBufferWindowMemory(k=10)

chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [98]:
chain.invoke(questions[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'input': 'Write me a detailed scenario for a Nashville songwriter.',
 'history': '',
 'response': "As an intelligence analyst, I have been tasked with providing a detailed scenario for a Nashville songwriter. Here is my report:\n\nSubject: Detailed Scenario for Nashville Songwriter\n\nIntroduction:\nThe subject of this report is a successful Nashville songwriter who has been experiencing a decline in creativity and inspiration in recent months. Our objective is to provide a detailed scenario that will help the subject to identify the sources of their creative blockage and find ways to overcome it.\n\nBackground:\nThe subject is a 35-year-old male songwriter who has been active in the Nashville music scene for the past decade. He has written several hit songs for various artists and has a reputation for being one of the most talented and innovative songwriters in the city. However, over the past year, he has noticed a significant decline in his creativity and has struggled to come up w

In [99]:
chain.run(questions[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Based on the scenario, one person I might meet in the BlueBird Cafe is the subject's collaborator, a fellow songwriter who has worked with the subject in the past. This person could be a valuable source of insight and support for the subject, as they have likely witnessed firsthand the challenges he is facing and can provide a different perspective on his creative blockage.\n\nAs an intelligence analyst, I would approach this person with an open mind and a willingness to listen to their thoughts and experiences. I would ask questions such as:\n\n* What have you noticed about the subject's creative process and how has it changed over time?\n* Have you observed any changes in the subject's behavior or mood that may be related to his creative blockage?\n* How do you think the subject could overcome his creative blockage and return to producing high-quality songs?\n\nBy gathering information from this person, I can gain a better understanding of the subject's strengths, weaknesses, and po

In [115]:
shakespeare_text = os.path.join("/content","drive","My Drive", "models", "pg100.txt")
os.path.exists(shakespeare_text)


True

In [116]:
loader = TextLoader(shakespeare_text,
                    encoding="utf8")
documents = loader.load()

In [117]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [118]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [119]:
from langchain.vectorstores import Chroma

In [109]:
!pip install chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [120]:
vectordb = Chroma.from_documents(documents=all_splits,
                                 embedding=embeddings,
                                 persist_directory="chroma_db")

In [121]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [122]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [123]:
query = "Who are the characters in MUCH ADO ABOUT NOTHING? Summarize. Keep it under 200 words."
test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: Who are the characters in MUCH ADO ABOUT NOTHING? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 23.592 sec.

Result:  

Characters:

* Claudio, a young nobleman from Florence
* Hero, Claudio's bride
* Benedick, a young nobleman and Claudio's friend
* Beatrice, Benedick's cousin and a witty and stubborn young lady
* Don John, Claudio's illegitimate brother and a deceitful villain
* Conrade, Don John's follower
* Borachio, a Sexton and Don John's follower

Summary:
In Much Ado About Nothing, Claudio and Hero fall in love and get married, while Benedick and Beatrice engage in a series of witty battles that disguise their true feelings for each other. The villainous Don John seeks to ruin the wedding by tricking Claudio into believing that Hero is unchaste. In the end, the misunderstandings are resolved, and the couples are reconciled.


In [124]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: Who are the characters in MUCH ADO ABOUT NOTHING? Summarize. Keep it under 200 words.
Retrieved documents: 4
Source:  /content/drive/My Drive/models/pg100.txt
Text:  ACT III
 Scene I. A Room in the Palace
 Scene II. The Forest of Arden
 Scene III. Another part of the Forest
 Scene IV. Another part of the Forest. Before a Cottage
 Scene V. Another part of the Forest

 ACT IV
 Scene I. The Forest of Arden
 Scene II. Another part of the Forest
 Scene III. Another part of the Forest

 ACT V
 Scene I. The Forest of Arden
 Scene II. Another part of the Forest
 Scene III. Another part of the Forest
 Scene IV. Another part of the Forest
 Epilogue


Dramatis Personæ

ORLANDO, youngest son of Sir Rowland de Boys
OLIVER, eldest son of Sir Rowland de Boys
JAQUES DE BOYS, second son of Sir Rowland de Boys
ADAM, Servant to Oliver
DENNIS, Servant to Oliver

ROSALIND, Daughter of Duke Senior
CELIA, Daughter of Duke Frederick
TOUCHSTONE, a Clown

DUKE SENIOR (Ferdinand), living in exile

JAQUES,